## Import libraries

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Load model

In [2]:
from tensorflow.keras.models import model_from_json

model_filepath = "./saved_models/resnet50.json"
weights_filepath = "./saved_models/resnet_ckpt3_epoch_04_val_loss_0.608.weights.h5"

json_file = open(model_filepath, 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

# load weights into new model
model.load_weights(weights_filepath)
print("Loaded model from disk")

Loaded model from disk


In [3]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,539,531 (169.90 MB)

 Trainable params: 6,983,939 (26.64 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 13,967,880 (53.28 MB)

## Load dataset

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '../chest-xray-pneumonia/Data_blance/Train'
test_dir = '../chest-xray-pneumonia/Data_blance/Test'
val_dir = '../chest-xray-pneumonia/Data_blance/Val'

img_size = 224
batch_size = 32
channels = 3   # 3 channels for color images

train_datagen = ImageDataGenerator(
    rescale = 1./255.0,
    zca_whitening = False,
    horizontal_flip = True,
    vertical_flip = False,
    shear_range = 0.2,
    zoom_range = 0.2
)

val_test_datagen = ImageDataGenerator(rescale=1./255.0)

train_set = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical',
)

test_set = val_test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    shuffle=False,
    class_mode='categorical',
)

val_set = val_test_datagen.flow_from_directory(
    directory=val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    shuffle=False,
    class_mode='categorical',
)

Found 3900 images belonging to 3 classes.
Found 300 images belonging to 3 classes.
Found 279 images belonging to 3 classes.


## Continue training

In [ ]:
# Get last evaluation result
scores_val = model.evaluate(val_set)
print("Val Loss:", scores_val[0])
print("Val Accuracy:", scores_val[1])

In [5]:
from sklearn.metrics import f1_score

y_pred_probs = model.predict(val_set)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = val_set.classes

current_f1 = f1_score(y_true, y_pred, average='macro')
print(current_f1)

c:\Users\HP\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step
0.695407531466488


In [6]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from custom_callback import F1Checkpoint

os.makedirs('saved_models', exist_ok=True)
checkpoint_filepath = "saved_models/resnet_ckpt4_epoch_{epoch:02d}_val_loss_{val_loss:.3f}.weights.h5"
checkpoint_callback = F1Checkpoint(
    filepath=checkpoint_filepath,
    validation_data=val_set,
    monitor='val_f1_score',
    mode='max',
    verbose=1,
    save_best_only=True,
    save_weights_only=True
)

checkpoint_callback.best_f1 = current_f1

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose = 1)

In [7]:
hist = model.fit(
  x=train_set, 
  epochs=10, 
  validation_data=val_set,
  callbacks=[checkpoint_callback, early_stopping_callback]
  # class_weight=class_weights_dict,
)

Epoch 1/10


c:\Users\HP\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6741 - loss: 0.7064 - precision: 0.7071 - recall: 0.6183

KeyboardInterrupt: 

## Plot Loss & Accuracy

In [ ]:
# Extract the training and validation loss values from the history object
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']

# Create a list of epoch numbers (1 to number of epochs)
epochs = range(1, len(train_loss) + 1)

# Plot the loss graph
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Extract the training and validation accuracy values from the history object
train_accuracy = hist.history['accuracy']
val_accuracy = hist.history['val_accuracy']

# Create a list of epoch numbers (1 to number of epochs)
epochs = range(1, len(train_accuracy) + 1)

# Plot the accuracy graph
plt.plot(epochs, train_accuracy , label='Training Acc')
plt.plot(epochs, val_accuracy, label='Validation Acc')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
scores_train = model.evaluate(train_set)

In [ ]:
scores_test = model.evaluate(test_set)

In [ ]:
scores_val = model.evaluate(val_set)

In [ ]:
# Print evaluation scores
print("Train Loss:", scores_train[0])
print("Train Accuracy:", scores_train[1])
print("Test Loss:", scores_test[0])
print("Test Accuracy:", scores_test[1])
print("Validation Loss:", scores_val[0])
print("Validation Accuracy:", scores_val[1])

In [ ]:
y_pred_probs = model.predict(test_set)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_set.classes

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

report = classification_report(y_true, y_pred, target_names=val_set.class_indices)
print(report)

In [ ]:
# Display confusion matrix using seaborn heatmap
conf_matrix = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=train_set.class_indices, yticklabels=train_set.class_indices)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()